In [ ]:
import pandas as pd
import os
import requests
import re
import json


In [ ]:
main_path = '../gsm8k/qwen_25_pred/'
files = os.listdir(main_path)
len(files)

In [ ]:

if '.DS_Store' in files:
    files.remove('.DS_Store')

len(files)

In [ ]:
api_key = ""

In [ ]:
with open(main_path + files[2001], 'r') as f:
        data = json.load(f)

In [ ]:
data

In [ ]:
for j, file_name in enumerate(files):
    print(j)
    if j<1262:
        continue
    with open(main_path + file_name, 'r') as f:
        data = json.load(f)
        
    reason_steps = []
    reason = data['Reason']
    for step_num, step in enumerate(reason):
        if step == 'Reason: [':
            continue
        elif step == ']':
            continue
        else:
            reason_steps.append(step)
    
    q = data['Question']
    answer = data['Answer']
    GT_answer = data['GT_answer']
    GT_reason = data['GT_reason']    
    generated_questions = []

    prompt = []

    # System prompt (single string, not list)
    prompt.append({
        "role": "system",
        "content": """You are given ONLY the step-by-step reasoning and calculations that a model produced for a math problem.

    Do THREE things and return EXACTLY one JSON object with this schema:
    {
      "question1": string,
      "question2": string,
      "numbers": [
        { "surface": string, "value": number|string, "unit": string|null, "role": string }
      ],
      "ops": [ "add" | "subtract" | "multiply" | "divide" | "percent" | "rate" | "ratio" | "average" | "area" | "time" | "distance" | "other" ],
      "composed_question": string
    }

    Instructions:
    1) Write two clear questions ("question1", "question2") using ONLY facts explicitly present in the reasoning. Do NOT include any answers.
    2) Extract ALL numeric facts explicitly present in the reasoning into "numbers". Normalize:
       - Percent -> decimal (e.g., 10% -> 0.10) in "value"
       - Word numbers -> digits
       - Fractions may remain strings in "value"
       Keep exact appearance in "surface". Include "unit" if present, else null. "role" is a short noun (e.g., initial_cost, sale_amount, rate, count, area, time, coefficient, other).
    3) List the operations seen in the reasoning in "ops" (e.g., add, subtract).
    4) Build "composed_question" using ONLY the numbers found in "numbers" and the operations in "ops". Do NOT invent any new numbers or units. Use generic nouns like "a person/item/buyer" if needed. Do NOT include any answer.

    Output JSON only. No extra text. No markdown."""
    })

    # Few-shot 1 — user
    prompt.append({
        "role": "user",
        "content": "REASONING:\nstep 1: John initially spends $1200 on a gaming PC.\nstep 2: He then sells the old video card for $300, reducing the net cost of his expenses.\nstep 3: After selling the old card, John buys a new video card for $500, increasing his total expenditure.\nstep 4: To find the net amount spent, we subtract the money received from selling the old card ($300) from the initial cost of the PC ($1200) and add the cost of the new video card ($500).\nstep 5: The calculation is: $1200 (initial PC cost) + $500 (new video card cost) - $300 (money from selling old card) = $1400."
    })

    # Few-shot 1 — assistant (JSON as string)
    prompt.append({
        "role": "assistant",
        "content": """{
          "question1": "John buys a gaming PC for $1200, sells the old video card for $300, and buys a new video card for $500. How much money did he spend on his computer after accounting for the sale?",
          "question2": "A person spends $1200 on a PC, sells a component for $300, and purchases a replacement for $500. What is the total net expenditure?",
          "numbers": [
            { "surface": "$1200", "value": 1200, "unit": "USD", "role": "initial_cost" },
            { "surface": "$300",  "value": 300,  "unit": "USD", "role": "sale_amount" },
            { "surface": "$500",  "value": 500,  "unit": "USD", "role": "purchase_amount" }
          ],
          "ops": ["add", "subtract"],
          "composed_question": "A person buys a PC for $1200, sells an old part for $300, and buys a new part for $500. After accounting for the sale and the purchase, what is the total net amount of money spent?"
        }"""
        })

        # Few-shot 2 — user
    prompt.append({
            "role": "user",
            "content": "REASONING:\nstep 1: Calculate the total cost of the land Carlson bought. The first piece of land cost $8000 and the second piece of land cost $4000. Therefore, the total cost is $8000 + $4000 = $12000.\nstep 2: Determine the total area of the land he bought. Since the cost of the land is $20 per square meter, the total area of the land he bought is $12000 / $20 per square meter = 600 square meters.\nstep 3: Add the area of the land he bought to the area of the land he initially owned. The initial area was 300 square meters, so the total area now is 300 square meters + 600 square meters = 900 square meters."
        })

        # Few-shot 2 — assistant
    prompt.append({
        "role": "assistant",
        "content": """{
      "question1": "Carlson initially owns 300 square meters of land. He buys more land costing $8000 and $4000 at a price of $20 per square meter. What is his total land area after the purchase?",
      "question2": "A buyer already has 300 square meters. He purchases additional land with costs $8000 and $4000 at a rate of $20 per square meter. Find the new total area of his land.",
      "numbers": [
        { "surface": "$8000", "value": 8000, "unit": "USD", "role": "cost" },
        { "surface": "$4000", "value": 4000, "unit": "USD", "role": "cost" },
        { "surface": "$20 per square meter", "value": 20, "unit": "USD per square meter", "role": "rate" },
        { "surface": "300 square meters", "value": 300, "unit": "square meters", "role": "area" }
      ],
      "ops": ["add", "divide", "add"],
      "composed_question": "A buyer owns 300 square meters of land. He purchases additional land costing $8000 and $4000 at a price of $20 per square meter. What is his total land area after the purchase?"
    }"""
    })

    # Final runtime example — user
    prompt.append({
        "role": "user",
         "content": "REASONING:\n" + str(reason_steps)
    })

    payload = {
                    "model": "gpt-3.5-turbo",
                    "messages": prompt,
                    "max_tokens": 4000,
                    "temperature": 0.2
                }
    headers = {
                    "Content-Type": "application/json",
                    "Authorization": f"Bearer {api_key}"
                }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    output = response.json()['choices'][0]['message']['content']
    json_str = re.sub(r"^```json\s*|```$", "", output.strip(), flags=re.DOTALL)
    split = json_str.split('\n')
    temp = split[1:-1]    
    
    for each in temp:
        generated_questions.append(each.strip())
        
    result = {
        'question':q,
        'answer':answer,
        'GT_answer':GT_answer,
        'GT_reason':GT_reason,
        'reason':reason_steps,
        'generated_question':generated_questions
    }
    path = '../gsm8k/generate_questions/' + file_name
    with open(path, 'w') as f:
        json.dump(result, f, indent = 2)
     
    

In [ ]:
response